# ANAYLSIS TEMPLATE
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [15]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""

!!! INPUT TEAMS HERE !!!

wrapped in quotes, eg.

home_side = "Arsenal"
away_side = "Everton"

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

"""
home_side = "Wolverhampton Wanderers"
away_side = "Ipswich"
matchday = "2024-12-14"

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Wolverhampton Wanderers v. Ipswich


### Calculate Projected Goals

In [16]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.5714285714285714 2.571428571428572
1.1428571428571428 2.142857142857143
1.63 1.38
0.9640666082383874 1.3146362839614374 0.8281573498964804 1.8633540372670814

 ================== projected_goals ==================== 

HOME Wolverhampton Wanderers projected goals: 2.0658570176536872
AWAY Ipswich projected goals:  2.1295474711623785


### Poisson Distribution of Goal Probabilities

In [17]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Wolverhampton Wanderers v. Ipswich [[1.50646477e-02 3.11214083e-02 3.21461898e-02 2.21364773e-02
  1.14326992e-02 4.72366439e-03 1.62640254e-03 4.79987871e-04
  1.23948289e-04]
 [3.20808825e-02 6.62745163e-02 6.84568373e-02 4.71406792e-02
  2.43464758e-02 1.00592676e-02 3.46350141e-03 1.02215696e-03
  2.63953765e-04]
 [3.41588811e-02 7.05673643e-02 7.28910423e-02 5.01941571e-02
  2.59234879e-02 1.07108439e-02 3.68784534e-03 1.08836588e-03
  2.81051037e-04]
 [2.42476530e-02 5.00921840e-02 5.17416450e-02 3.56302801e-02
  1.84017661e-02 7.60308351e-03 2.61781390e-03 7.72575603e-04
  1.99503842e-04]
 [1.29091320e-02 2.66684210e-02 2.75465723e-02 1.89690932e-02
  9.79685859e-03 4.04778182e-03 1.39368974e-03 4.11309106e-04
  1.06213225e-04]
 [5.49812189e-03 1.13583337e-02 1.17323467e-02 8.07911690e-03
  4.17257509e-03 1.72398871e-03 5.93585694e-04 1.75180453e-04
  4.52372211e-05]
 [1.95141859e-03 4.03135180e-03 4.16409820e-03 2.86747716e-03
  1.48094945e-03 6.11885965e-04 2.10678152e-04 6.21

### Under Market

In [18]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Wolverhampton Wanderers v. Ipswich
    Goals      Prob  Implied Odds
0    0.5  0.015065     66.380576
1    1.5  0.078267     12.776787
2    2.5  0.210847      4.742786
3    3.5  0.396255      2.523628


In [19]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Wolverhampton Wanderers v. Ipswich
    Goals      Prob  Implied Odds
0    0.5  0.984263      1.015989
1    1.5  0.921061      1.085705
2    2.5  0.788481      1.268261
3    3.5  0.603073      1.658175


### 1 x 2 Probability

In [20]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
                         winner      prob  implied_odds
0  Wolverhampton Wanderers (1)  0.386880      2.584779
1                     Draw (x)  0.201612      4.960017
2                  Ipswich (2)  0.410835      2.434067


# Full Results Summary:

In [ ]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):
    if string == "Ipswich":
        return "Ipswich Town"
    else:
        return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
filtered_odds = odds_df.loc[odds_df["match"] == match_string]
implied = {
    "1": 1 / filtered_odds["1"],
    "x": 1 / filtered_odds["x"],
    "2": 1 / filtered_odds["2"],
    "point": filtered_odds["ovr_und_point"],
    "over": 1 / filtered_odds["over"],
    "under": 1 / filtered_odds["under"]
}
implied_odds_df = pd.DataFrame(implied)

print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)

CALCULATED PROBABILITIES FOR Wolverhampton Wanderers v. Ipswich Town


        ~ ~ moneyline market ~ ~

                         winner      prob  implied_odds
0  Wolverhampton Wanderers (1)  0.386880      2.584779
1                     Draw (x)  0.201612      4.960017
2                  Ipswich (2)  0.410835      2.434067


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.984263      1.015989
1    1.5  0.921061      1.085705
2    2.5  0.788481      1.268261
3    3.5  0.603073      1.658175


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.015065     66.380576
1    1.5  0.078267     12.776787
2    2.5  0.210847      4.742786
3    3.5  0.396255      2.523628

 ~~ MyBookie Odds for Wolverhampton Wanderers v. Ipswich Town ~~
       1    x    2  ovr_und_point  over  under
2  1.93  3.7  3.6            2.5  1.69   2.08

 ~~ MyBookie Odds IMPLIED for Wolverhampton Wanderers v. Ipswich Town ~~
           1        x         2  point  

# Conclusion and Pick

SAFE PICK: Our model suggests that 'Over 2.5' is undervalued. My Bookie odds of 1.69 implies a 59% chance of hitting, where we calculate a 78% chance.

RISKIER PICK: Moneyline is almost always more risky because we're betting on 3 outcomes instead of 2. Our model suggests that Ipswich is slightly more likely to win that Wolves at 41% to 38%. MyBookie's odds consider an Ipswich win to have a 27% chance of hitting. So we consider an Ipswich win to be the value pick *if we decide this game is winnable for Ipswich...*

### My Pick:

**O 2.5 @ 1.69**